In [231]:
import pandas, numpy, pathlib

from skops.io import load

from sklearn.utils import resample

import matplotlib.pyplot as plt
import seaborn

from misc import construct_line

from statsmodels.stats.weightstats import ztest as ztest

pathlib.Path('pdf/figure-5').mkdir(exist_ok=True)

number_of_bootstraps = 2

### Model validation on `validation` and `mic` sets

As discussed in the manuscript we have two further datasets (`validation` & `mic`) to evaluate the models on.

First let's load the trained models, the datasets and the results for Suspect-PZA


In [232]:
best_model = {}

for model in ['LR', 'NN', 'XB']:
    best_model[model] = load('models/'+model.lower()+'.skops', trusted=True)

X={}
Y={}
Z={}

for i in ['validation-samples', 'validation-mutations', 'mic']:
    X[i]={}
    Y[i]={}
    Z[i]={}
    with open('data/ds-'+i+'.npy', 'rb') as f:
        Y[i]['input'] = numpy.load(f)
        X[i]['input'] = numpy.load(f)
        Z[i]['input'] = numpy.load(f, allow_pickle=True)

# load in the results for SuspectPZA
suspectpza={}
for i in ['validation-samples', 'validation-mutations','mic']:
    suspectpza[i]={}
    with open('data/suspectpza-'+i+'.npy', 'rb') as f:
        suspectpza[i]['input'] = numpy.load(f)
        suspectpza[i]['predicted'] = numpy.load(f)
        suspectpza[i]['muts'] = numpy.load(f, allow_pickle=True)
line = []

In [233]:
X['validation-mutations']['input'].shape

(199, 14)

For comparison later, let's make a subset of `validation-samples` that exclude those mutations which have an inconsistent phenotype, either because too few are in the dataset or they do not consistently test as R or S, suggesting they may be near the breakpoint/ECOFF. 

We need to do this for both the `validation-samples` features and the SuspectPZA results.

In [234]:
df = pandas.read_csv('data/ds-validation-mutations-full.csv')

mask = ~numpy.isin(Z['validation-samples']['input'], df[df.CONSISTENT_PHENOTYPE=='U'].MUTATION.unique())

X['validation-samplesnoU']={}
Y['validation-samplesnoU']={}
Z['validation-samplesnoU']={}

X['validation-samplesnoU']['input']=X['validation-samples']['input'][mask]
Y['validation-samplesnoU']['input']=Y['validation-samples']['input'][mask]
Z['validation-samplesnoU']['input']=Z['validation-samples']['input'][mask]

mask = ~numpy.isin(suspectpza['validation-samples']['muts'],(df[df.CONSISTENT_PHENOTYPE=='U'].MUTATION.unique()))
suspectpza['validation-samplesnoU']={}
suspectpza['validation-samplesnoU']['input']=suspectpza['validation-samples']['input'][mask]
suspectpza['validation-samplesnoU']['predicted']=suspectpza['validation-samples']['predicted'][mask]
suspectpza['validation-samplesnoU']['muts']=suspectpza['validation-samples']['muts'][mask]

In [251]:
df[df.CONSISTENT_PHENOTYPE!='U'].MUTATION.unique()

array(['A102R', 'A102T', 'A134V', 'A143G', 'A143T', 'A143V', 'A146E',
       'A146P', 'A146T', 'A146V', 'A171V', 'A30V', 'A3E', 'A46E', 'A46T',
       'A79T', 'A79V', 'C138R', 'C14R', 'C72R', 'C72Y', 'D129Y', 'D12A',
       'D12E', 'D12N', 'D136Y', 'D49A', 'D49G', 'D49N', 'D63A', 'D8A',
       'D8E', 'D8G', 'D8N', 'E15G', 'F106S', 'F13I', 'F13V', 'F58V',
       'F81C', 'F81S', 'F94C', 'F94S', 'G105D', 'G105V', 'G108R', 'G124D',
       'G132A', 'G132C', 'G132D', 'G132S', 'G162D', 'G17D', 'G17S',
       'G24D', 'G97C', 'G97D', 'G97R', 'G97S', 'G97V', 'H137P', 'H137Q',
       'H43P', 'H43Y', 'H51D', 'H51P', 'H51Q', 'H51R', 'H51Y', 'H57D',
       'H57L', 'H57P', 'H57Q', 'H57R', 'H57Y', 'H71P', 'H71Q', 'H71Y',
       'H82D', 'H82R', 'I31S', 'I5S', 'I5T', 'I6L', 'I6S', 'I6T', 'I6V',
       'I90T', 'K48N', 'K96E', 'K96N', 'K96R', 'K96T', 'L116R', 'L120P',
       'L120Q', 'L120R', 'L151S', 'L156P', 'L159R', 'L172P', 'L172R',
       'L182F', 'L19P', 'L27P', 'L35R', 'L4S', 'L85P', 'L85R', 'M175I

In [236]:
Y['validation-mutations']

{'input': array([0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0,
        1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
        1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
        1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        0])}

The below function takes a supplied model, applies it to the features of the different datasets and measures a range of metrics we can use to evaluate their performance.

Note that this uses the separate `construct_line` function which can be found in `misc.py`.

In [237]:
def validate_model(line, best_model, model_name, X, Y):

    for dataset in ['validation-samples', 'validation-samplesnoU', 'validation-mutations', 'mic']: 
        
        Y[dataset]['predicted'] = best_model.predict(X[dataset]['input'])
        Y[dataset]['scores'] = best_model.predict_proba(X[dataset]['input'])[:,1]

        row = construct_line(model_name , dataset, None, Y[dataset], None)
        line.append(row)

    return(line)

In [238]:
for model in ['NN', 'XB', 'LR']:
    line = validate_model(line, best_model[model], model, X, Y)

In [239]:
for i in ['validation-samples', 'validation-samplesnoU', 'validation-mutations', 'mic']:
    line.append(construct_line('SP', i, None, suspectpza[i], None))

In [240]:
def bootstrap_model(line, best_model, model_name, X, Y):
    x={}
    y={}
    for dataset in ['validation-samples', 'validation-samplesnoU', 'validation-mutations', 'mic']:
        print(model_name, dataset)
        for i in range(number_of_bootstraps):
            x['input'],y['input'] = resample(X[dataset]['input'], Y[dataset]['input'])
            y['predicted'] = best_model.predict(x['input'])
            y['scores'] = best_model.predict_proba(x['input'])[:,1]
            row = construct_line(model_name, dataset+'_'+str(i), None, y, None)
            line.append(row)
    return(line)

In [241]:
for model in ['LR', 'NN', 'XB']:
    line = bootstrap_model(line, best_model[model], model, X, Y)

LR validation-samples
LR validation-samplesnoU
LR validation-mutations
LR mic
NN validation-samples
NN validation-samplesnoU
NN validation-mutations
NN mic
XB validation-samples
XB validation-samplesnoU
XB validation-mutations
XB mic


In [242]:
test_results = pandas.DataFrame(line, columns=['model', 'dataset', 'sensitivity_mean', 'sensitivity_std', 'specificity_mean', 'specificity_std' ,'roc_auc_mean', 'roc_auc_std','TN','FP','FN','TP', 'model_parameters'])
test_results[:3]

,model,dataset,sensitivity_mean,sensitivity_std,specificity_mean,specificity_std,roc_auc_mean,roc_auc_std,TN,FP,FN,TP,model_parameters
0,NN,validation-samples,90.517241,None,51.649236,None,73.731156,None,642,601,264,2520,null
1,NN,validation-samplesnoU,91.479239,None,66.666667,None,80.821306,None,538,269,197,2115,null
2,NN,validation-mutations,94.193548,None,52.272727,None,77.565982,None,23,21,9,146,null


In [243]:
line = []
for i in ['validation-samples', 'validation-samplesnoU', 'validation-mutations', 'mic']:
    for model in ['LR', 'NN', 'XB']:
        sens_mean = test_results[(test_results.model==model) & (test_results.dataset!=i) & (test_results.dataset.str.contains(i+"_"))].sensitivity_mean.mean()
        sens_std = 1.96*test_results[(test_results.model==model) & (test_results.dataset!=i) & (test_results.dataset.str.contains(i+"_"))].sensitivity_mean.std()/(number_of_bootstraps**0.5)
        spec_mean = test_results[(test_results.model==model) & (test_results.dataset!=i) & (test_results.dataset.str.contains(i+"_"))].specificity_mean.mean()
        spec_std = 1.96*test_results[(test_results.model==model) & (test_results.dataset!=i) & (test_results.dataset.str.contains(i+"_"))].specificity_mean.std()/(number_of_bootstraps**0.5)
        roc_mean = test_results[(test_results.model==model) & (test_results.dataset!=i) & (test_results.dataset.str.contains(i+"_"))].roc_auc_mean.mean()
        row = [model, 'bootstrapped-'+i, sens_mean, sens_std, spec_mean, spec_std, roc_mean, None, None, None, None, None, None ]
        line.append(row)

extra_rows = pandas.DataFrame(line, columns=['model', 'dataset', 'sensitivity_mean', 'sensitivity_std', 'specificity_mean', 'specificity_std' ,'roc_auc_mean', 'roc_auc_std','TN','FP','FN','TP', 'model_parameters'])
test_results  = pandas.concat([test_results, extra_rows])
test_results[:3]

,model,dataset,sensitivity_mean,sensitivity_std,specificity_mean,specificity_std,roc_auc_mean,roc_auc_std,TN,FP,FN,TP,model_parameters
0,NN,validation-samples,90.517241,None,51.649236,None,73.731156,None,642,601,264,2520,null
1,NN,validation-samplesnoU,91.479239,None,66.666667,None,80.821306,None,538,269,197,2115,null
2,NN,validation-mutations,94.193548,None,52.272727,None,77.565982,None,23,21,9,146,null


In [244]:
extra_rows


,model,dataset,sensitivity_mean,sensitivity_std,specificity_mean,specificity_std,roc_auc_mean,roc_auc_std,TN,FP,FN,TP,model_parameters
0,LR,bootstrapped-validation-samples,97.949817,0.332518,39.573584,3.071517,79.813823,None,None,None,None,None,None
1,NN,bootstrapped-validation-samples,90.402880,0.318457,50.126129,2.756013,72.940454,None,None,None,None,None,None
2,XB,bootstrapped-validation-samples,96.920144,0.092886,45.210647,2.341381,80.939237,None,None,None,None,None,None
3,LR,bootstrapped-validation-samplesnoU,98.536402,0.238349,56.998434,2.693446,85.245002,None,None,None,None,None,None
4,NN,bootstrapped-validation-samplesnoU,91.860829,0.081310,66.980133,1.799195,80.938711,None,None,None,None,None,None
5,XB,bootstrapped-validation-samplesnoU,97.271187,0.150869,61.534842,0.856499,86.314386,None,None,None,None,None,None
6,LR,bootstrapped-validation-mutations,97.711253,0.740722,49.319728,3.333333,85.080029,None,None,None,None,None,None
7,NN,bootstrapped-validation-mutations,94.079963,2.042297,59.059233,3.756098,79.086468,None,None,None,None,None,None
8,XB,bootstrapped-validation-mutations,98.323000,2.046414,62.476548,1.838649,88.400217,None,None,None,None,None,None
9,LR,bootstrapped-mic,100.000000,0.000000,0.000000,0.000000,61.704545,None,None,None,None,None,None


In [245]:
test_results[(test_results.model=='XB') & (test_results.dataset!='validation-samplesnoU') & (test_results.dataset.str.contains('validation-samplesnoU'+"_"))]

,model,dataset,sensitivity_mean,sensitivity_std,specificity_mean,specificity_std,roc_auc_mean,roc_auc_std,TN,FP,FN,TP,model_parameters
34,XB,validation-samplesnoU_0,97.194213,None,61.097852,None,86.169707,None,512,326,64,2217,null
35,XB,validation-samplesnoU_1,97.348161,None,61.971831,None,86.459065,None,484,297,62,2276,null


In [246]:
for dataset in ['validation-samples', 'validation-samplesnoU', 'validation-mutations', 'mic']:
    for metric in ['sensitivity_mean', 'specificity_mean']:
        for i in ['XB', 'NN', 'LR']:
            for j in ['XB', 'NN', 'LR']:
                if i!=j:
                    a = numpy.array(test_results[(test_results.model==i)& (test_results.dataset.str.contains(dataset+'_'))][metric])
                    b = numpy.array(test_results[(test_results.model==j)& (test_results.dataset.str.contains(dataset+'_'))][metric])
                    assert len(a) == number_of_bootstraps
                    tstat, pvalue = ztest(a,b)
                    if pvalue < 0.05:
                        print(dataset,metric, i,j,"Signficant",pvalue)
        print()

validation-samples sensitivity_mean XB NN Signficant 0.0
validation-samples sensitivity_mean XB LR Signficant 5.0494560027008465e-09
validation-samples sensitivity_mean NN XB Signficant 0.0
validation-samples sensitivity_mean NN LR Signficant 1.8262827799357194e-226
validation-samples sensitivity_mean LR XB Signficant 5.0494560027008465e-09
validation-samples sensitivity_mean LR NN Signficant 1.8262827799357194e-226

validation-samples specificity_mean XB NN Signficant 0.007718550825701561
validation-samples specificity_mean XB LR Signficant 0.004226515918235587
validation-samples specificity_mean NN XB Signficant 0.007718550825701561
validation-samples specificity_mean NN LR Signficant 5.387648863348632e-07
validation-samples specificity_mean LR XB Signficant 0.004226515918235587
validation-samples specificity_mean LR NN Signficant 5.387648863348632e-07

validation-samplesnoU sensitivity_mean XB NN Signficant 0.0
validation-samplesnoU sensitivity_mean XB LR Signficant 1.48160339526002

/Users/fowler/Library/Python/3.11/lib/python/site-packages/statsmodels/stats/weightstats.py:748: RuntimeWarning: invalid value encountered in scalar divide
  zstat = (value1 - value2 - diff) / std_diff


In [247]:
# test_results = pandas.DataFrame(line, columns=['model', 'dataset', 'sensitivity_mean', 'sensitivity_std', 'specificity_mean', 'specificity_std' ,'roc_auc_mean', 'roc_auc_std','TN','FP','FN','TP', 'model_parameters'])

# calculate the diagnostic odds ration
# test_results['diagnostic_odds_ratio_mean'] = (test_results['TN']*test_results['TP'])/(test_results['FN']*test_results['FP'])
test_results['diagnostic_odds_ratio_mean'] = 0
test_results['diagnostic_odds_ratio_std'] = 0

# save to disc as a CSV
test_results.to_csv('results-validation.csv', index=False)

test_results[:20]

,model,dataset,sensitivity_mean,sensitivity_std,specificity_mean,specificity_std,roc_auc_mean,roc_auc_std,TN,FP,FN,TP,model_parameters,diagnostic_odds_ratio_mean,diagnostic_odds_ratio_std
0,NN,validation-samples,90.517241,None,51.649236,None,73.731156,None,642,601,264,2520,null,0,0
1,NN,validation-samplesnoU,91.479239,None,66.666667,None,80.821306,None,538,269,197,2115,null,0,0
2,NN,validation-mutations,94.193548,None,52.272727,None,77.565982,None,23,21,9,146,null,0,0
3,NN,mic,94.000000,None,42.857143,None,63.142857,None,3,4,3,47,null,0,0
4,XB,validation-samples,97.198276,None,46.017699,None,80.722592,None,572,671,78,2706,null,0,0
5,XB,validation-samplesnoU,97.621107,None,63.073110,None,86.880823,None,509,298,55,2257,null,0,0
6,XB,validation-mutations,97.419355,None,59.090909,None,88.284457,None,26,18,4,151,null,0,0
7,XB,mic,100.000000,None,14.285714,None,67.714286,None,1,6,0,50,null,0,0
8,LR,validation-samples,98.060345,None,40.305712,None,79.642377,None,501,742,54,2730,null,0,0
9,LR,validation-samplesnoU,98.442907,None,56.133829,None,84.841520,None,453,354,36,2276,null,0,0


In [248]:
test_results.model.value_counts()

model
NN    16
XB    16
LR    16
SP     4
Name: count, dtype: int64

In [249]:
test_results[test_results.model=='XB']

,model,dataset,sensitivity_mean,sensitivity_std,specificity_mean,specificity_std,roc_auc_mean,roc_auc_std,TN,FP,FN,TP,model_parameters,diagnostic_odds_ratio_mean,diagnostic_odds_ratio_std
4,XB,validation-samples,97.198276,None,46.017699,None,80.722592,None,572,671,78,2706,null,0,0
5,XB,validation-samplesnoU,97.621107,None,63.073110,None,86.880823,None,509,298,55,2257,null,0,0
6,XB,validation-mutations,97.419355,None,59.090909,None,88.284457,None,26,18,4,151,null,0,0
7,XB,mic,100.000000,None,14.285714,None,67.714286,None,1,6,0,50,null,0,0
32,XB,validation-samples_0,96.872753,None,44.016064,None,80.022506,None,548,697,87,2695,null,0,0
33,XB,validation-samples_1,96.967535,None,46.405229,None,81.855968,None,568,656,85,2718,null,0,0
34,XB,validation-samplesnoU_0,97.194213,None,61.097852,None,86.169707,None,512,326,64,2217,null,0,0
35,XB,validation-samplesnoU_1,97.348161,None,61.971831,None,86.459065,None,484,297,62,2276,null,0,0
36,XB,validation-mutations_0,99.367089,None,63.414634,None,87.017598,None,26,15,1,157,null,0,0
37,XB,validation-mutations_1,97.278912,None,61.538462,None,89.782836,None,32,20,4,143,null,0,0


In [250]:
test_results[(test_results.model=='XB') & (test_results.dataset.str.contains('validation-samples'))]

,model,dataset,sensitivity_mean,sensitivity_std,specificity_mean,specificity_std,roc_auc_mean,roc_auc_std,TN,FP,FN,TP,model_parameters,diagnostic_odds_ratio_mean,diagnostic_odds_ratio_std
4,XB,validation-samples,97.198276,None,46.017699,None,80.722592,None,572,671,78,2706,null,0,0
5,XB,validation-samplesnoU,97.621107,None,63.073110,None,86.880823,None,509,298,55,2257,null,0,0
32,XB,validation-samples_0,96.872753,None,44.016064,None,80.022506,None,548,697,87,2695,null,0,0
33,XB,validation-samples_1,96.967535,None,46.405229,None,81.855968,None,568,656,85,2718,null,0,0
34,XB,validation-samplesnoU_0,97.194213,None,61.097852,None,86.169707,None,512,326,64,2217,null,0,0
35,XB,validation-samplesnoU_1,97.348161,None,61.971831,None,86.459065,None,484,297,62,2276,null,0,0
2,XB,bootstrapped-validation-samples,96.920144,0.092886,45.210647,2.341381,80.939237,None,None,None,None,None,None,0,0
5,XB,bootstrapped-validation-samplesnoU,97.271187,0.150869,61.534842,0.856499,86.314386,None,None,None,None,None,None,0,0
